In [22]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, LassoCV, RidgeCV, ElasticNetCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, FunctionTransformer, Imputer
import warnings
def ignore_warn(*args, **kwargs):
    pass
from scipy import stats
from scipy.stats import norm, skew
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

#read in both

ames_data = pd.read_csv("train.csv") 
test = pd.read_csv('test.csv')

# #ames_data.columns = ames_data.columns.map(lambda x: str(x.replace(' ', '_')))

# #missing_data = ames_data.isnull().sum()

# fig, ax = plt.subplots(figsize=(15,15))
# missing_data.plot(kind = 'bar')

# ames_data = ames_data.drop(["Alley",'Fireplace_Qu','Pool_QC','Fence', 'Misc_Feature'], axis = 1)

# missing_data = ames_data.isnull().sum()

# fig, ax = plt.subplots(figsize=(13,13))
# missing_data.plot(kind = 'bar')

In [23]:
all_data = pd.concat((ames_data, test)).reset_index(drop=True)
all_data.drop(['SalePrice'], axis=1, inplace=True)
numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index
#all_data.columns = all_data.columns.map(lambda x: str(x.replace(' ', '_')))

# Check the skew of all numerical features
skewed_feats = all_data[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
print("\nSkew in numerical features: \n")
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness.head(10)

skewness = skewness[abs(skewness) > 0.75]
print("There are {} skewed numerical features to Box Cox transform".format(skewness.shape[0]))

from scipy.special import boxcox1p
skewed_features = skewness.index
lam = 0.15
for feat in skewed_features:
    #all_data[feat] += 1
    all_data[feat] = boxcox1p(all_data[feat], lam)


Skew in numerical features: 

There are 38 skewed numerical features to Box Cox transform


In [24]:
all_data.columns = all_data.columns.map(lambda x: str(x.replace(' ', '_')))

In [25]:
#all_data = ames_data.dropna(axis = 0, how = 'any')

In [26]:
#numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index

# Check the skew of all numerical features
skewed_feats = all_data[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
print("\nSkew in numerical features: \n")
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness.head(10)

KeyError: "Index(['1st Flr SF', '2nd Flr SF', '3Ssn Porch', 'Bedroom AbvGr',\n       'Bsmt Full Bath', 'Bsmt Half Bath', 'Bsmt Unf SF', 'BsmtFin SF 1',\n       'BsmtFin SF 2', 'Enclosed Porch', 'Full Bath', 'Garage Area',\n       'Garage Cars', 'Garage Yr Blt', 'Gr Liv Area', 'Half Bath',\n       'Kitchen AbvGr', 'Lot Area', 'Lot Frontage', 'Low Qual Fin SF',\n       'MS SubClass', 'Mas Vnr Area', 'Misc Val', 'Mo Sold', 'Open Porch SF',\n       'Overall Cond', 'Overall Qual', 'Pool Area', 'Screen Porch',\n       'TotRms AbvGrd', 'Total Bsmt SF', 'Wood Deck SF', 'Year Built',\n       'Year Remod/Add', 'Yr Sold'],\n      dtype='object') not in index"

In [16]:
all_data["Total_SF"] = all_data['Gr_Liv_Area'] + all_data["Total_Bsmt_SF"] + all_data["Garage_Area"] + all_data['Wood_Deck_SF']\
    + all_data["Open_Porch_SF"] + all_data['Enclosed_Porch'] + all_data['3Ssn_Porch'] + all_data['Screen_Porch']\
    + ames_data['Pool_Area']

all_data["Total_Bathrooms"] = all_data['Bsmt_Full_Bath'] + all_data['Bsmt_Half_Bath'] + all_data['Full_Bath'] + all_data['Half_Bath']

# all_data['Quality_Average_5'] = (all_data['Exter_Qual'] + all_data['Exter_Cond'] + all_data['Bsmt_Qual'] + all_data['Bsmt_Cond'] + all_data['Bsmt_Exposure'] + all_data['BsmtFin_Type_1'] + all_data['BsmtFin_Type_2'] + all_data['Heating_QC'] + all_data['Kitchen_Qual'] + all_data['Garage_Cond'] + all_data['Garage_Qual']).mean()

# all_data['Quality_Average_10'] = (all_data["Overall_Qual"] + all_data['Overall_Cond']).mean()

In [30]:
all_data = pd.get_dummies(all_data)

In [31]:
all_data.shape

(1556, 265)